In [16]:
import gensim
from gensim.models import Word2Vec

In [7]:
from google.cloud import bigquery
import pandas as pd
import ast
import glove_helper
import tensorflow as tf
import numpy as np
import scipy
import keras
from itertools import groupby
from os.path import basename, splitext
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import regex as re
import ast
import glove_helper
import tensorflow as tf
import numpy as np
import scipy

from itertools import groupby
from os.path import basename, splitext
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

Using TensorFlow backend.


In [8]:
client = bigquery.Client(project='manifest-frame-203601')

/Users/cathyzhou/anaconda3/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/.
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [9]:
QUERY = (
    """
    select distinct repo_path,c_content from w266_final.final_20k
    """)
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

df = []
for row in rows:
    df.append([row.repo_path,row.c_content])

In [10]:
df = pd.DataFrame(df)
df.columns = ['repo_path','content']
df.shape

(172413, 2)

In [11]:
def cleanup(docstring_list):
    
    """takes a list of doc strings and converts to a single flat list of tokens"""
    
    tokens = [tf.keras.preprocessing.text.text_to_word_sequence(i) for i in docstring_list]
    flat_tokens = [item for sublist in tokens for item in sublist]
    flat_string = " ".join(flat_tokens)
    
    return flat_string

def get_docstrings(source):
    
    """function to walk through parse tree and return list of docstrings"""
    
    NODE_TYPES = {
    ast.ClassDef: 'Class',
    ast.FunctionDef: 'Function/Method',
    ast.Module: 'Module'
    }
    
    docstrings = []
    
    try:
        tree = ast.parse(source)
    except:
        return " "
       
    for node in ast.walk(tree):
        if isinstance(node, tuple(NODE_TYPES)):
            docstring = ast.get_docstring(node)
            docstrings.append(docstring)
    
    docstrings =  [x for x in docstrings if x is not None]
    clean_string = cleanup(docstrings)
            
    return clean_string

In [12]:
df['docstrings'] = [get_docstrings(x) for x in list(df['content'])]
doc = []
for i in df['docstrings']:
    doc.append([i.split()])
df['docstring2'] = doc
df.head()

,repo_path,content,docstrings,docstring2
0,ilogue/niprov tests/test_mediumviewer.py,#!/usr/bin/python\n# -*- coding: UTF-8 -*-\nfr...,,[[]]
1,UndeadMastodon/Loltris Matrix.py,#!/usr/bin/python2\n#-*- coding: utf-8 -*-\n\n...,prints a matrix to the console for debugging p...,"[[prints, a, matrix, to, the, console, for, de..."
2,anhstudios/swganh data/scripts/templates/objec...,#### NOTICE: THIS FILE IS AUTOGENERATED\n#### ...,,[[]]
3,Southpaw-TACTIC/TACTIC src/bin/example/add_fla...,import sys\r\nimport tacticenv\r\n\r\n\r\nfrom...,,[[]]
4,scrapinghub/exporters exporters/writers/dropbo...,from collections import Counter\nfrom exporter...,writes items to dropbox folder options availab...,"[[writes, items, to, dropbox, folder, options,..."


In [29]:
def create_embedding_matrix(model):
    # convert the wv word vectors into a numpy matrix that is suitable for insertion
    # into our TensorFlow and Keras models
    embedding_matrix = np.zeros((len(model.wv.vocab), 100))
    for i in range(len(model.wv.vocab)):
        embedding_vector = model.wv[model.wv.index2word[i]]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix




In [30]:
embed = []
vector_dim = 100
for i in df['docstring2']:
    #print(i[0])
 #   print(i)
    if len(i[0]) ==0:
        embed.append([[[0]]])
    else:
        model = Word2Vec(i, sg=1,iter=10, min_count=1, size=100, workers=4)
        embedding_matrix = create_embedding_matrix(model)
        embed.append([embedding_matrix])

In [31]:
df['doc_string_embed']=embed

In [47]:
df.head(10)

,repo_path,content,docstrings,docstring2,doc_string_embed
0,ilogue/niprov tests/test_mediumviewer.py,#!/usr/bin/python\n# -*- coding: UTF-8 -*-\nfr...,,[[]],[[[0]]]
1,UndeadMastodon/Loltris Matrix.py,#!/usr/bin/python2\n#-*- coding: utf-8 -*-\n\n...,prints a matrix to the console for debugging p...,"[[prints, a, matrix, to, the, console, for, de...","[[[-0.0015476032858714461, 0.00098642380908131..."
2,anhstudios/swganh data/scripts/templates/objec...,#### NOTICE: THIS FILE IS AUTOGENERATED\n#### ...,,[[]],[[[0]]]
3,Southpaw-TACTIC/TACTIC src/bin/example/add_fla...,import sys\r\nimport tacticenv\r\n\r\n\r\nfrom...,,[[]],[[[0]]]
4,scrapinghub/exporters exporters/writers/dropbo...,from collections import Counter\nfrom exporter...,writes items to dropbox folder options availab...,"[[writes, items, to, dropbox, folder, options,...","[[[-0.0005623144679702818, 0.00231970450840890..."
5,astrofrog/sedfitter sedfitter/convolve/monochr...,"from __future__ import print_function, divisio...",convolve all the model seds in a model directo...,"[[convolve, all, the, model, seds, in, a, mode...","[[[-0.0003378069377504289, 0.00475694937631487..."
6,materialsproject/pymatgen pymatgen/core/tests/...,# coding: utf-8\n# Copyright (c) Pymatgen Deve...,testing xcfunc api,"[[testing, xcfunc, api]]","[[[-0.00021332580945454538, -0.000282575783785..."
7,dnerdy/namesync namesync/providers/cloudflare.py,import json\n\nfrom namesync.exceptions import...,,[[]],[[[0]]]
8,kamyu104/LeetCode Python/path-sum-ii.py,from __future__ import print_function\n# Time:...,,[[]],[[[0]]]
9,beiko-lab/gengis bin/Lib/site-packages/numpy/d...,\r\n# http://www.pgroup.com\r\n\r\nfrom numpy....,,[[]],[[[0]]]


In [49]:
def embed_search(words):
    words_l = words.split()
    model = Word2Vec(words_l, sg=1, min_count=1, size=100, workers=4)
    embedding_matrix = create_embedding_matrix(model)
    return (embedding_matrix)

In [37]:
def find_nn(words, embeddings):
    
    search = embed_search(words)
    distances = [scipy.spatial.distance.cosine(search[0], i[0][0][0]) for i in embeddings]
    nn = np.argsort(np.asarray(distances))
    
    return nn

In [40]:
def top_n_code(search_terms, docstrings, embeddings, n):
    
    top_n = find_nn(search_terms, embeddings)[0:n]
    code = [df['content'].iloc[i] for i in top_n]
    
    return code
    #return top_n
    
#search = "model for LSTM network"
doc_strings = list(df['docstrings'])
embed_vecs = list(df['doc_string_embed'])

#print(top_n_code(search, doc_strings, embed_vecs, 5))

In [45]:
search1 = "model for LSTM network"
query1 = (top_n_code(search1, doc_strings, embed_vecs, 5))
print(query1[0])

/Users/cathyzhou/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


# -*- coding: utf-8 -*-
"""
/***************************************************************************
    Useful network functions
                             -------------------
    begin            : 2011-03-01
    copyright        : (C) 2011 by Luiz Motta
    author           : Luiz P. Motta
    email            : motta _dot_ luiz _at_ gmail.com
 ***************************************************************************/

/***************************************************************************
 *                                                                         *
 *   This program is free software; you can redistribute it and/or modify  *
 *   it under the terms of the GNU General Public License as published by  *
 *   the Free Software Foundation; either version 2 of the License, or     *
 *   (at your option) any later version.                                   *
 *                                                                         *
 **************************

In [44]:
search2 = "train a model for image recognition"
query2 = (top_n_code(search2, doc_strings, embed_vecs, 5))
print(query1[0])

/Users/cathyzhou/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


# -*- coding: utf-8 -*-
"""
/***************************************************************************
    Useful network functions
                             -------------------
    begin            : 2011-03-01
    copyright        : (C) 2011 by Luiz Motta
    author           : Luiz P. Motta
    email            : motta _dot_ luiz _at_ gmail.com
 ***************************************************************************/

/***************************************************************************
 *                                                                         *
 *   This program is free software; you can redistribute it and/or modify  *
 *   it under the terms of the GNU General Public License as published by  *
 *   the Free Software Foundation; either version 2 of the License, or     *
 *   (at your option) any later version.                                   *
 *                                                                         *
 **************************